### Get surface temperature information for the models ###

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import matplotlib
from matplotlib.ticker import MaxNLocator,FormatStrFormatter
import pickle
from glob import glob
import sys  
import matplotlib.gridspec as gridspec
from scipy.stats import linregress
from dateutil.relativedelta import relativedelta
import itertools
import pickle
# from cartopy.util import add_cyclic_point


from scipy.ndimage import label,find_objects
import scipy.ndimage as ndimage
from matplotlib.colors import LinearSegmentedColormap

import datetime as dt
# import cartopy.crs as ccrs
import matplotlib.ticker as mticker
# from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import xarray as xr
from scipy.stats import norm


%matplotlib inline

### Get names of all models you want to plot ###
Creating two separate lists for historical and SSP runs.<br>


Specify model name here. The code will attempt to extract both SSP and HIST versions.<br>
Also we are only looking at the surface air temperature: `tas`, so make sure that it is available.

In [24]:
model_name={}
model_name['HIST']='NESM3'
model_name['SSP']='@NESM3'

In [31]:
keys=['HIST','SSP']

direc={key: None for key in keys}

direc['HIST']='/neelin2020/CMIP6-HISTORICAL/'
direc['SSP']='/neelin2020/CMIP6-SSP585/'

ta_dir={'HIST':'tas','SSP':'tas'}  # change the directory name between ta and tas as required 
ta_ext={'HIST':'tas_3hr','SSP':'tas_3hr'}

### Set temperature variable name ###
TA_VAR={'HIST':'tas','SSP':'ta'}


model_dir={key: [] for key in keys}
model_list={key: [] for key in keys}

model_index={key: None for key in keys}

for key in keys:
    print(key)
    diri=direc[key]
    list_temp=(glob(diri+'*'))

    for name in list_temp:
        temp_name=name.split('/')[-1]
        model_list[key].append(temp_name)
        model_dir[key].append(diri+temp_name+'/'+ta_dir[key]+'/'+ta_ext[key]+'*')


#     model_list[key]=model_list[key][1:] ### neglect the first name, since it is not really a model
#     model_dir[key]=model_dir[key][1:] ### neglect the first name, since it is not really a model
    print(model_list[key])
    model_index[key]=model_list[key].index(model_name[key]) ## find index of any model you want    

    


HIST
['@BCC-CSM2-MR', '@FGOALS-f3-L', '@GISS-E2-1-G', 'ACCESS-CM2', 'ACCESS-ESM1-5', 'BCC-CSM2-MR', 'CMIP6-yxli.tar', 'auto_folder.ncl', 'wget_multi.ncl', 'CESM2', 'CNRM-CM6-1-HR', 'CNRM-CM6-1', 'FGOALS-g3', 'GFDL-CM4', 'IITM-ESM', 'KACE-1-0-G', 'MIROC-ES2L', 'MIROC6', 'MPI-ESM-1-2-HAM', 'MPI-ESM1-2-LR', 'MRI-ESM2-0', 'NESM3', 'NorESM2-LM', 'NorESM2-MM', 'MPI-ESM1-2-HR', 'CanESM5', 'SAM0-UNICON', '@IPSL-CM6A-LR', 'IPSL-CM6A-LR', 'GISS-E2-1-G']
SSP
['ACCESS-CM2', '20200713_redo_uncomplete_wget_multi.ncl', '20200817_restart_download_wget_multi.ncl', 'auto_folder.ncl', '@KACE-1-0-G', '@NESM3', 'sh_file_backup.tar', 'wget_multi.ncl', '@NorESM2-LM', '@NorESM2-MM', 'CNRM-CM6-1-HR', 'CNRM-CM6-1', 'FGOALS-g3', 'GFDL-CM4', 'IPSL-CM6A-LR', 'MIROC6', 'MRI-ESM2-0', 'BCC-CSM2-MR']


In [42]:
model_dir[key][model_index[key]]
!ls /neelin2020/CMIP6-SSP585/@NESM3/tas/

tas_3hr_NESM3_ssp585_r1i1p1f1_gn_201501010130-205012312230.nc
tas_3hr_NESM3_ssp585_r1i1p1f1_gn_205101010130-207512312230.nc
tas_3hr_NESM3_ssp585_r1i1p1f1_gn_207601010130-210012312230.nc


In [34]:
### fix datetime format ##
def fix_datetime(ds):

    try:
        datetimeindex = ds.indexes['time'].to_datetimeindex()
        ds['time'] = datetimeindex
    except:
        pass


model_files={key:None for key in keys}
ta_ds={key:None for key in keys}
ta_global_mean={key:None for key in keys}


for key in keys:
    model_files[key]=(glob(model_dir[key][model_index[key]]))
    ta_ds[key]=xr.open_mfdataset(model_files[key])
    fix_datetime(ta_ds[key])
    ta_ds[key].coords['lon'] = (ta_ds[key].coords['lon'] + 180) % 360  - 180
    ta_ds[key] = ta_ds[key].sortby(ta_ds[key].lon)



In [44]:
model_dir[key][model_index[key]]

'/neelin2020/CMIP6-SSP585/@NESM3/tas/tas_3hr*'

### Examine the data time range use it set time bounds for extraction##
The cell below gives starting and ending year and month

In [35]:
for key in keys:
    print(ta_ds[key].time[0].dt.year.values,ta_ds[key].time[0].dt.month.values)
    print(ta_ds[key].time[-1].dt.year.values,ta_ds[key].time[-1].dt.month.values)
    print('------------------')

1979 1
2014 12
------------------
2015 1
2100 12
------------------


Make sure to match the temperature time range <b>exactly</b> to the precip. cluster stats range

In [36]:
strt_date={key:None for key in keys}
end_date={key:None for key in keys}

## Specify start and end dates to extract from for SSP
strt_date['SSP']=dt.datetime(2091,1,1)
end_date['SSP']=dt.datetime(2100,12,31)

## and HIST
strt_date['HIST']=dt.datetime(2005,1,1)
end_date['HIST']=dt.datetime(2014,12,31)

for key in keys:
    
    print(key)
    time_slice=slice(strt_date[key],end_date[key])

    ## Specify latitudinal bounds to extract
    lat_slice=slice(-30,30)
    
    ta_ds_slice=ta_ds[key].sel(time=time_slice,lat=lat_slice) ## only extract specified timeslice
    ta_global_mean[key]=ta_ds_slice.mean(dim=('time','lat','lon')).compute()
    
    lon=ta_ds_slice.lon
    lat=ta_ds_slice.lat

# dates=ta_ds.time.dt.strftime('%Y-%M-%d %H')

HIST
SSP


Create timestamps which will act as co-ordinates for the mean temperature

In [37]:
time_stamp={key:None for key in keys}
for key in keys:
    time_stamp[key]="%s_%s"%(strt_date[key].strftime("%Y%m"),end_date[key].strftime("%Y%m"))

### Create and store the file as a netcdf ###

 Create file information<br>
 We will store the information as a netcdf 

In [38]:
ta_array=xr.DataArray(
             data=np.array((ta_global_mean['HIST'].tas.values,ta_global_mean['SSP'].tas.values)),
             dims=["time"],
             coords=dict(time=['HIST: %s'%(time_stamp['HIST']),'SSP: %s'%(time_stamp['SSP'])]),
             attrs=dict(description="Average Surface Air temperature.",
             units="K"),)

### Set file name including directory ###

In [6]:
dname='./files/'
fname='%s_tas_ave_%s-%s_%s-%s.nc'%(model_name['HIST'],strt_date['HIST'].strftime("%Y"),end_date['HIST'].strftime("%Y"),
                                  strt_date['SSP'].strftime("%Y"),end_date['SSP'].strftime("%Y"))
ta_array.to_netcdf(dname+fname)

NameError: name 'model_name' is not defined

### Reading temperature info from the netcdf files ###

In [60]:
### enter full path of the netcdf file you want to read
file_name='./files/CNRM-CM6-1_tas_ave_2005-2014_2091-2100.nc'
ds=xr.open_dataarray(file_name)
ds.time

<xarray.DataArray 'time' (time: 2)>
array(['HIST: 200501_201412', 'SSP: 209101_210012'], dtype=object)
Coordinates:
  * time     (time) object 'HIST: 200501_201412' 'SSP: 209101_210012'

### Method one to read the temperatures ###

Note the coordinate names `HIST: 200501_201412` and `SSP: 209101_210012`. Use these as indices

In [64]:
tas_hist=ds.sel(time='HIST: 200501_201412').values
tas_ssp=ds.sel(time='SSP: 209101_210012').values
print(tas_hist,tas_ssp)

297.6228 301.5965


### Method two to read the temperatures ##

Alternatively use the knowledge that the first value is HIST and the second value is SSP

In [62]:
tas_hist,tas_ssp=ds.values[0],ds.values[1]
print(tas_hist, tas_ssp)

297.6228 301.5965
